In [1]:
import torch
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense

# Load the pre-trained PyTorch model
model = torch.hub.load('pytorch/vision:v0.9.0', 'squeezenet1_0', pretrained=True)

# Define the Keras model
keras_model = Sequential()
keras_model.add(InputLayer(input_shape=(3, 224, 224)))
for i, (name, layer) in enumerate(model.named_children()):
    if isinstance(layer, torch.nn.Conv2d):
        keras_layer = Conv2D(layer.out_channels, layer.kernel_size, strides=layer.stride, padding=layer.padding)
        weights = layer.state_dict()
        keras_layer.set_weights([weights['weight'].numpy(), weights.get('bias', np.zeros(layer.out_channels)).numpy()])
        keras_model.add(keras_layer)
    elif isinstance(layer, torch.nn.MaxPool2d):
        keras_model.add(MaxPooling2D(pool_size=layer.kernel_size, strides=layer.stride, padding=layer.padding))
    elif isinstance(layer, torch.nn.ReLU):
        keras_model.add(Flatten())
    elif isinstance(layer, torch.nn.Linear):
        keras_layer = Dense(layer.out_features)
        weights = layer.state_dict()
        keras_layer.set_weights([weights['weight'].numpy().T, weights.get('bias', np.zeros(layer.out_features)).numpy()])
        keras_model.add(keras_layer)

# Test the Keras model
x_test = np.random.randn(1, 3, 224, 224)
y_pred = keras_model.predict(x_test)
print(y_pred)


Using cache found in C:\Users\Scorpio/.cache\torch\hub\pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to C:\Users\Scorpio/.cache\torch\hub\checkpoints\squeezenet1_0-a815701f.pth
100.0%


[[[[-0.6512918   0.46607423  0.83468574 ...  0.19057867  0.6533083
    -0.914362  ]
   [ 0.65300584 -0.13832593 -0.07369771 ...  0.00792651 -0.81954205
    -0.69041115]
   [-1.3666917   0.29977235  0.28130236 ...  0.94256544  1.0405385
    -0.7172916 ]
   ...
   [ 1.2025074   0.94053155 -0.74547946 ...  2.6588373   0.46428797
     0.6899495 ]
   [ 0.7546218   0.51618034 -0.9098455  ... -1.6476227   0.86903435
     0.21496135]
   [ 1.4733369   0.69526476  0.50338167 ... -0.23325467 -0.519604
     1.2241831 ]]

  [[-0.93967    -0.4525363   1.5264649  ...  0.3364583   0.04406328
     0.9431964 ]
   [ 0.22613157 -1.1250004   2.4493527  ... -0.15801634 -0.6164469
    -0.53339016]
   [-0.20424962  0.5775373  -0.00803954 ...  0.48673216 -1.4523306
     0.26151925]
   ...
   [ 0.17992687  1.130787    0.4298042  ...  1.1080565   0.73804116
     0.8292625 ]
   [ 1.3049074   1.4865134  -0.47835866 ... -2.362526    0.9000299
    -0.3919159 ]
   [ 1.274588    0.3937143   0.9311822  ... -0.33819535 

In [2]:
keras_model.save("SqueezeNet.h5")

In [3]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [1]:
import torch
import numpy as np
import torchvision.models as models

# Load the pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

# Save the weights in channel-last format
weights = {}
for name, param in model.named_parameters():
    if "bias" in name:
        # Save biases as usual
        weights[name] = param.detach().cpu().numpy()
    else:
        # Permute weight tensor from channel-first to channel-last format
        weights[name] = param.permute(2, 3, 1, 0).detach().cpu().numpy()

np.savez("weights.npz", **weights)


c:\Users\Scorpio\miniconda3\envs\torch2keras\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Scorpio\miniconda3\envs\torch2keras\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to C:\Users\Scorpio/.cache\torch\hub\checkpoints\squeezenet1_1-b8a52dc0.pth
100.0%


In [2]:
import torch
import torchvision.models as models

# Load the SqueezeNet model
model = models.squeezenet1_0(pretrained=True)

weights = {}
for name, param in model.named_parameters():
    if "bias" in name:
        # Save biases as usual
        weights[name] = param.detach().cpu().numpy()
    else:
        # Permute weight tensor from channel-first to channel-last format
        weights[name] = param.permute(2, 3, 1, 0).detach().cpu().numpy()

np.savez("weights.npz", **weights)

# Print the model's layers
for name, module in model.named_children():
    print(name, module)


features Sequential(
  (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (3): Fire(
    (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
    (squeeze_activation): ReLU(inplace=True)
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1_activation): ReLU(inplace=True)
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (expand3x3_activation): ReLU(inplace=True)
  )
  (4): Fire(
    (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
    (squeeze_activation): ReLU(inplace=True)
    (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (expand1x1_activation): ReLU(inplace=True)
    (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (expand3x3_activation): ReLU(inplace=True)
  )
  (5): Fire(
    (squeeze): Conv2d(128, 32, kernel_size=(1, 1), stride=(1

c:\Users\Scorpio\miniconda3\envs\torch2keras\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torchsummary import summary

summary(model, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 109, 109]          14,208
              ReLU-2         [-1, 96, 109, 109]               0
         MaxPool2d-3           [-1, 96, 54, 54]               0
            Conv2d-4           [-1, 16, 54, 54]           1,552
              ReLU-5           [-1, 16, 54, 54]               0
            Conv2d-6           [-1, 64, 54, 54]           1,088
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]           9,280
              ReLU-9           [-1, 64, 54, 54]               0
             Fire-10          [-1, 128, 54, 54]               0
           Conv2d-11           [-1, 16, 54, 54]           2,064
             ReLU-12           [-1, 16, 54, 54]               0
           Conv2d-13           [-1, 64, 54, 54]           1,088
             ReLU-14           [-1, 64,

In [4]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.layers import BatchNormalization

def fire_module(x, fire_id, squeeze, expand):
    s_id = 'fire' + str(fire_id) + '/squeeze1x1'
    e1_id = 'fire' + str(fire_id) + '/expand1x1'
    e3_id = 'fire' + str(fire_id) + '/expand3x3'

    x = Conv2D(squeeze, (1, 1), padding='valid', name=s_id)(x)
    x = Activation('relu', name=s_id + '/relu')(x)

    left = Conv2D(expand, (1, 1), padding='valid', name=e1_id)(x)
    left = Activation('relu', name=e1_id + '/relu')(left)

    right = Conv2D(expand, (3, 3), padding='same', name=e3_id)(x)
    right = Activation('relu', name=e3_id + '/relu')(right)

    x = Concatenate(axis=3, name='fire' + str(fire_id) + '/concat')([left, right])
    return x


def SqueezeNet(input_shape=(224, 224, 3), classes=1000):
    input_tensor = Input(shape=input_shape)

    x = Conv2D(96, (7, 7), strides=(2, 2), padding='same', name='conv1')(input_tensor)
    x = Activation('relu', name='conv1/relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)

    x = Dropout(0.5, name='drop9')(x)

    x = Conv2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='conv10/relu')(x)

    x = GlobalAveragePooling2D(name='avgpool10')(x)
    output_tensor = Activation('softmax', name='softmax')(x)

    model = Model(input_tensor, output_tensor, name='squeezenet')
    return model

keras_model= SqueezeNet()
keras_model.summary()


Model: "squeezenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 96  14208       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/relu (Activation)        (None, 112, 112, 96  0           ['conv1[0][0]']                  
                                )                                                        

In [10]:
# Load the PyTorch weights in channel-last format
weights = np.load('weights.npz')

In [6]:
keras_model.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 96  14208       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/relu (Activation)        (None, 112, 112, 96  0           ['conv1[0][0]']                  
                                )                                                        

In [7]:
already_parsed=[]


def count_parameters_pytorch(model,target):
    total_params = 0
    counter = 0
    for name, parameter in model.named_parameters():
        params = parameter.numel()
        total_params+=params
        counter = counter + 1 
        if (counter >= 2 ):
            if (total_params == target and name not in already_parsed ) :
                already_parsed.append(name)
                return str(name[0:-5])
                # x = 5
            else :
                total_params = 0
                counter = 0

print(already_parsed)

[]


In [8]:
for layer in keras_model.layers :
    if layer.count_params() != 0 :
        torch_layer_name = count_parameters_pytorch(model, layer.count_params())
        print(layer.count_params())
        layer.set_weights([weights[torch_layer_name +'.weight'], weights[torch_layer_name +'.bias']])

14208
1552
1088
9280
2064
1088
9280
4128
4224
36992
8224
4224
36992
12336
9408
83136
18480
9408
83136
24640
16640
147712
32832
16640
147712
513000


In [9]:
keras_model.save("SqueezeNet.h5")



In [11]:
# Generate a random input tensor
input_tensor = torch.randn(1, 3, 224, 224)

# Convert the tensor to channel-last format
input_tensor_keras = input_tensor.permute(0, 2, 3, 1)



In [15]:
# Use the PyTorch model to predict the output for the input tensor
with torch.no_grad():
    pytorch_output = model(input_tensor).numpy()

# Use the Keras model to predict the output for the input tensor
keras_output = keras_model.predict(np.transpose(input_tensor.numpy(), (0, 2, 3, 1)))

# Compare the two predictions
# print(np.allclose(pytorch_output, keras_output, rtol=1e-03, atol=1e-05))
print(np.argmax(pytorch_output))
print(np.argmax(keras_output))

735
828

